In [ ]:
from query.datasets.prelude import *

In [ ]:
video = Video.objects.get(path='tvnews/videos/CNNW_20160702_180000_CNN_Newsroom_With_Fredricka_Whitfield.mp4')
speakers = list(Speaker.objects.filter(video=video, labeler__name='lium').order_by('min_frame'))

In [ ]:
#video.extract_audio(output_path='/tmp/out.wav')

In [ ]:
def fmt_time(t):
    return '{:02d}:{:02d}:{:02d}.{:03d}'.format(
        int(t / 3600), int(t / 60 % 60), int(t % 60), int(t * 1000 % 1000))

def cut_audio(seg):
    sp.check_call('ffmpeg -y -i {} -ss {} -t {} /tmp/seg.wav'.format(
        '/tmp/out.wav',
        fmt_time(seg.min_frame / video.fps),
        fmt_time((seg.max_frame - seg.min_frame) / video.fps)),
        shell=True)

In [ ]:
lines = readlines('/app/deps/speaker-diarization/stdout')
labeler, _ = Labeler.objects.get_or_create(name='aalto')
unk_gender = Gender.objects.get(name='U')
identities = {}
speakers = []
for line in lines:
    parts = [s.split('=') for s in line.split(' ')]
    dct = {p[0]: p[1] for p in parts}
    if not dct['speaker'] in identities:
        identities[dct['speaker']] = Identity(name=dct['speaker'])
    speakers.append(Speaker(
        video=video,
        min_frame=int(float(dct['start-time']) * video.fps),
        max_frame=int(float(dct['end-time']) * video.fps),
        labeler=labeler,
        gender=unk_gender,
        identity=identities[dct['speaker']]))

In [ ]:
#Identity.objects.all().delete()
Speaker.objects.filter(labeler=labeler).delete()

In [ ]:
Identity.objects.bulk_create(identities.values())
#_ = Speaker.objects.bulk_create(speakers)

In [ ]:
for s in speakers:
    s.identity = s.identity
Speaker.objects.bulk_create(speakers)

In [ ]:
speakers[0].identity

In [ ]:
print(Speaker.objects.filter(labeler=labeler)[0].identity)